# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

res_sum = 0

for sqrt_value in generator:
    res_sum += sqrt_value
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


In [4]:
print(f"Sum of square roots up to {limit}: {res_sum}")

Sum of square roots up to 5: 8.382332347441762


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [8]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [6]:
!pip install "dlt[duckdb]"

In [12]:
# load the people_1 to the duckdb
import dlt

p1 = dlt.pipeline(pipeline_name="people_1_pipeline", destination="duckdb", dataset_name="people_1")

info = p1.run(people_1(), table_name="people_1_data", write_disposition="replace")

print(info)

Pipeline people_1_pipeline load step completed in 0.15 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_1
The duckdb destination used duckdb:////Users/liuruizhu/Library/Mobile Documents/com~apple~CloudDocs/study_intern/de_project/workshop1/people_1_pipeline.duckdb location to store data
Load package 1712584164.382219 is LOADED and contains no failed jobs


In [13]:
p1.pipeline_name

'people_1_pipeline'

In [14]:
# show outcome

import duckdb

conn = duckdb.connect(f"{p1.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{p1.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_1_data table below:")

people_1_data = conn.sql("SELECT * FROM people_1_data").df()
display(people_1_data)

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1_data       │
└─────────────────────┘




 people_1_data table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1712584164.382219,ikvB0CI4asBO0A
1,2,Person_2,27,City_A,1712584164.382219,TfzEg2daZIGZvA
2,3,Person_3,28,City_A,1712584164.382219,PKD48bTRMwq4gA
3,4,Person_4,29,City_A,1712584164.382219,Vbz/y1cQZoaqAQ
4,5,Person_5,30,City_A,1712584164.382219,c12P8dCDcRdcDw


In [24]:
# load the people_2 to the duckdb
p2 = dlt.pipeline(pipeline_name="people_1_pipeline", destination="duckdb", dataset_name="people_1")

info = p2.run(people_2(), table_name="people_1_data", write_disposition="append")

print(info)

Pipeline people_1_pipeline load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_1
The duckdb destination used duckdb:////Users/liuruizhu/Library/Mobile Documents/com~apple~CloudDocs/study_intern/de_project/workshop1/people_1_pipeline.duckdb location to store data
Load package 1712625904.6305962 is LOADED and contains no failed jobs


In [25]:
# show outcome
conn = duckdb.connect(f"{p1.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{p1.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_1_data table below:")

people_1_data = conn.sql("SELECT * FROM people_1_data").df()
display(people_1_data)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1_data       │
└─────────────────────┘




 people_1_data table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712584164.382219,ikvB0CI4asBO0A,None
1,2,Person_2,27,City_A,1712584164.382219,TfzEg2daZIGZvA,None
2,3,Person_3,28,City_A,1712584164.382219,PKD48bTRMwq4gA,None
3,4,Person_4,29,City_A,1712584164.382219,Vbz/y1cQZoaqAQ,None
4,5,Person_5,30,City_A,1712584164.382219,c12P8dCDcRdcDw,None
5,3,Person_3,33,City_B,1712625904.6305962,OXuNQuhaFJl5aQ,Job_3
6,4,Person_4,34,City_B,1712625904.6305962,Wr+ccNi9IvNarw,Job_4
7,5,Person_5,35,City_B,1712625904.6305962,z2ohOmdgo+qJbA,Job_5
8,6,Person_6,36,City_B,1712625904.6305962,0jy9Iqxw4Jwy6A,Job_6
9,7,Person_7,37,City_B,1712625904.6305962,LK3ZYnzqf7F88g,Job_7


In [26]:
# method 1
# age_sum = people_1_data["age"].sum()

# method 2
age_sum = conn.sql("SELECT sum(age) FROM people_1_data")


print(age_sum)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘



# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [28]:
#Install the dependencies
!pip install dlt[duckdb]

zsh:1: no matches found: dlt[duckdb]


In [29]:
# to do: homework :)

# load the generator people_1 to the duckbd with primary key
people_pipeline_with_key = dlt.pipeline(pipeline_name="people_pipeline_with_key", destination="duckdb", dataset_name="people_with_key")

info = people_pipeline_with_key.run(people_1(), table_name="people_table_with_key", write_disposition="replace", primary_key="id")

print(info)

Pipeline people_pipeline_with_key load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_with_key
The duckdb destination used duckdb:////Users/liuruizhu/Library/Mobile Documents/com~apple~CloudDocs/study_intern/de_project/workshop1/people_pipeline_with_key.duckdb location to store data
Load package 1712627552.328615 is LOADED and contains no failed jobs


In [30]:
# show outcome
conn = duckdb.connect(f"{people_pipeline_with_key.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{people_pipeline_with_key.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_table_with_key table below:")

people_table = conn.sql("SELECT * FROM people_table_with_key").df()
display(people_table)

Loaded tables: 


┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ _dlt_loads            │
│ _dlt_pipeline_state   │
│ _dlt_version          │
│ people_table_with_key │
└───────────────────────┘




 people_table_with_key table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1712627552.328615,lXwuJyc4thW5tA
1,2,Person_2,27,City_A,1712627552.328615,aI0+p1ya/x6a6w
2,3,Person_3,28,City_A,1712627552.328615,++mda+uYi44JlA
3,4,Person_4,29,City_A,1712627552.328615,WbUikdUufxeuPw
4,5,Person_5,30,City_A,1712627552.328615,n6U4uybtsrM/7A


In [31]:
# load the generator people_2 to the duckbd with primary key
info = people_pipeline_with_key.run(people_2(), table_name="people_table_with_key", write_disposition="merge", primary_key="id")

print(info)

Pipeline people_pipeline_with_key load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_with_key
The duckdb destination used duckdb:////Users/liuruizhu/Library/Mobile Documents/com~apple~CloudDocs/study_intern/de_project/workshop1/people_pipeline_with_key.duckdb location to store data
Load package 1712627656.814548 is LOADED and contains no failed jobs


In [32]:
# show outcome
conn = duckdb.connect(f"{people_pipeline_with_key.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{people_pipeline_with_key.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_table_with_key table below:")

people_table = conn.sql("SELECT * FROM people_table_with_key").df()
display(people_table)

Loaded tables: 


┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ _dlt_loads            │
│ _dlt_pipeline_state   │
│ _dlt_version          │
│ people_table_with_key │
└───────────────────────┘




 people_table_with_key table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712627552.328615,lXwuJyc4thW5tA,None
1,2,Person_2,27,City_A,1712627552.328615,aI0+p1ya/x6a6w,None
2,5,Person_5,35,City_B,1712627656.814548,sq+udSi7pITPyQ,Job_5
3,4,Person_4,34,City_B,1712627656.814548,svZpp/Vcl0Vq7w,Job_4
4,8,Person_8,38,City_B,1712627656.814548,a7qeCPLLWkBe2w,Job_8
5,3,Person_3,33,City_B,1712627656.814548,i1HPPQI37XtRrg,Job_3
6,6,Person_6,36,City_B,1712627656.814548,4U+T2dBuYj/gQg,Job_6
7,7,Person_7,37,City_B,1712627656.814548,gkvPoBmcWlYMWw,Job_7


In [40]:
conn.sql("select sum(age) from people_table_with_key")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX